#### TODO>:
- Build a Bert Archiecture model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass

In [3]:
@dataclass
class EchoFinConfig:
    vocab_size:int = 17000
    dim:int = 256
    nheads:int = 4
    nlayers:int = 4
    maxlen = 128
    drop:bool= 0.25
    num_classes:int = 3
    
config = EchoFinConfig()

In [4]:
class EchoFinEmbeddings(nn.Module):
    def __init__(self, config: EchoFinConfig) -> None:
        super().__init__()
        self.tok_embeddings = nn.Embedding(config.vocab_size, config.dim)
        self.pos_embeddings = nn.Embedding(config.maxlen, config.dim)

    def forward(self, tok: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        bs, sq = tok.size()
        tok_embeds = self.tok_embeddings(tok)
        pos_tokens = torch.arange(sq, device=tok.device).repeat(bs, 1)
        if mask is not None:pos_tokens = pos_tokens * mask
        embeds = tok_embeds + self.pos_embeddings(pos_tokens)
        return embeds

In [5]:
class FeedForward(nn.Module):
    def __init__(self, config: EchoFinConfig) -> None:
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(config.dim, 4 * config.dim),
            nn.GELU(),
            nn.Dropout(config.drop),
            nn.Linear(4 * config.dim, config.dim)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:return self.mlp(x)

In [6]:
class EchoFinLayer(nn.Module):
    def __init__(self, config: EchoFinConfig) -> None:
        super().__init__()
        self.multi_self_attention = nn.MultiheadAttention(config.dim, num_heads=config.nheads, dropout=config.drop)
        self.norm = nn.LayerNorm(config.dim) 
        self.ffn = FeedForward(config)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        attn, _ = self.multi_self_attention(x, x, x)
        return self.ffn(self.norm(attn))

In [7]:
class EchoFin(nn.Module):
    def __init__(self, config: EchoFinConfig) -> None:
        super().__init__()
        self.embeddings = EchoFinEmbeddings(config)
        self.layers = nn.ModuleList([EchoFinLayer(config) for _ in range(config.nlayers)])
        self.norm = nn.LayerNorm(config.dim) 
        self.fc_layer = nn.Linear(config.dim, config.num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embeddings(x)
        for l in self.layers:x = l(x)
        logits = self.fc_layer(F.relu(self.norm(x[:, -1, :])))
        return logits

In [8]:
tokens = torch.randint(0, config.vocab_size, size=(4, config.maxlen))
model = EchoFin(config)
output = model(tokens)
print(output.shape)

torch.Size([4, 3])


#### Testing HuggingFace model

In [32]:
import warnings; warnings.filterwarnings("ignore")
import pandas as pd
from transformers import pipeline

from torchmetrics.classification import MulticlassAccuracy
from torch.utils.data import DataLoader, Dataset

In [12]:
data_path = r"..\data\finnews.csv"

In [79]:
data = pd.read_csv(data_path)

In [83]:
labels = {'neutral':0, 'positive':1, 'negative':2}
data['sentiment'] = data['sentiment'].map(labels)

In [41]:
class Dataset(Dataset):
    def __init__(self, x, y): self.x, self.y = x, y
    def __len__(self): return len(self.x)
    def __getitem__(self, idx): return self.x[idx], torch.tensor(self.y[idx], dtype=torch.float64)

In [43]:
tdset = Dataset(data['news'].tolist(), data['sentiment'].tolist())
test_dl = DataLoader(tdset, batch_size=128, shuffle=True)

In [ ]:
def get_predictions(pipeline, dl, device, step):
    predictions = []
    for i in range(0, len(dl), 128):
        lbl = pipeline(dl[i:i+step])
        predictions.extend([labels[h['label']] for h in lbl])
    return predictions

In [ ]:
fin_pipeline = pipeline("sentiment-analysis", model="ProsusAI/finbert")

In [80]:
predictions = get_predictions(fin_pipeline, data['news'].tolist(), "cpu", step=128)

In [84]:
predictions = torch.tensor(predictions)
targets = torch.tensor(data['sentiment'].tolist())

In [88]:
metric = MulticlassAccuracy(num_classes=3)
f"accuracy:, {metric(predictions, targets).item():.3f}"

'accuracy:, 0.942'